# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [ ]:
from typing import Any, Optional
from intergrax.globals.settings import GLOBAL_SETTINGS
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.llm_adapters.llm_usage_track import LLMUsageTracker
from intergrax.memory.conversational_memory import ConversationalMemory
from intergrax.tools.tools_agent import ToolsAgent, ToolsAgentConfig
from intergrax.tools.tools_base import ToolRegistry, ToolBase
from pydantic import BaseModel, Field

# ---------- SCHEMA ----------
class WeatherAnswer(BaseModel):
    """
    Structured representation of a weather query result.

    This model is used as `output_model` for the tools agent, allowing the LLM
    to return a validated, typed object instead of a free-form string.
    """
    city: str = Field(
        ...,
        description="City name for which the weather information applies. Example: 'Warsaw'.",
    )
    tempC: float = Field(
        ...,
        description="Current temperature in degrees Celsius for the specified city.",
    )
    summary: str = Field(
        ...,
        description=(
            "Short text summary of current weather conditions, "
            "e.g. 'partly cloudy', 'sunny', or 'light rain'."
        ),
    )

# ---------- TOOL ----------
class WeatherArgs(BaseModel):
    """
    Input schema for the WeatherTool.

    Fields:
      - city: name of the city to query the weather for.
    """
    city: str = Field(..., description="City name, e.g. 'Warsaw'")


class WeatherTool(ToolBase):
    """
    Simple weather tool returning demo data.

    Responsibilities:
      - Accept a city name via validated arguments (WeatherArgs).
      - Return a dictionary compatible with the WeatherAnswer schema.

    NOTE:
      This is a stub implementation. In production, integrate with a real
      weather API (e.g., OpenWeatherMap) and map API responses to the schema.
    """
    name = "get_weather"
    description = "Returns current weather for a city."
    schema_model = WeatherArgs

    def run(self, 
            run_id:Optional[str] = None,
            llm_usage_tracker: Optional[LLMUsageTracker] = None,
            **kwargs) -> Any:
        city = kwargs["city"]
        # Demo/static result, for testing the tools agent + output_model wiring.
        return {"city": city, "tempC": 12.3, "summary": "partly cloudy"}


# ---------- SETUP ----------
# Shared conversational memory for multi-turn interactions.
memory = ConversationalMemory()

# Tool registry that contains all callable tools available to the agent.
tools = ToolRegistry()
tools.register(WeatherTool())

# LLM used as planner/controller for tools and as a natural language generator.
# Here we use an Ollama-backed model exposed via LangChain's ChatOllama.
llm = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

# Tools agent orchestrating:
#   - LLM reasoning,
#   - automatic tool selection and invocation,
#   - mapping tool outputs into structured responses,
#   - optional use of conversational memory.
agent = ToolsAgent(
    llm=llm,
    tools=tools,
    memory=memory,
    config=ToolsAgentConfig(),    
)

# ---------- USAGE ----------
# Ask a natural-language question and request a structured result
# using the WeatherAnswer Pydantic model as `output_model`.
res = agent.run(
    "What's the weather like in Warsaw?",
    context=None,
    output_model=WeatherAnswer,
)

# Human-readable answer produced by the LLM (string).
print("[ANS]", res["answer"])

# Parsed, validated Pydantic object of type WeatherAnswer.
print("[OUTPUT_STRUCTURE]", res["output_structure"])

# The same object converted to a plain dictionary.
# Example: {'city': 'Warsaw', 'tempC': 12.3, 'summary': 'partly cloudy'}
print("[AS DICT]", res["output_structure"].model_dump())


[ANS] The current weather in Warsaw is partly cloudy with a temperature of 12.3 degrees Celsius.
[OUTPUT_STRUCTURE] city='Warsaw' tempC=12.3 summary='partly cloudy'
[AS DICT] {'city': 'Warsaw', 'tempC': 12.3, 'summary': 'partly cloudy'}


# Output structure + RAG

In [ ]:
from intergrax.rag.embedding_manager import EmbeddingManager
from intergrax.rag.rag_prompts import default_rag_system_instruction
from intergrax.rag.rag_retriever import RagRetriever
from intergrax.rag.vectorstore_manager import VectorstoreManager, VSConfig
from intergrax.rag.re_ranker import ReRanker, ReRankerConfig
from intergrax.rag.rag_answerer import AnswererConfig, RagAnswerer
from intergrax.memory.conversational_memory import ConversationalMemory
from dotenv import load_dotenv

# Load environment variables (useful when the LLM adapter or vector store
# expects configuration via environment, e.g., API keys, model names).
load_dotenv()

# --- (optional) Pydantic for structured output ---
# If Pydantic is not available, provide a minimal fallback so the script
# can still run (though structured output will be degraded).
try:
    from pydantic import BaseModel, Field
except Exception:
    class BaseModel:
        """Fallback BaseModel stub when Pydantic is not installed."""
        pass

    def Field(*a, **k):
        """Fallback Field stub doing nothing when Pydantic is not installed."""
        return None  # no-op placeholder


# -------------------------
# SCHEMA for structured output
# -------------------------
class ExecSummary(BaseModel):
    """
    Compact, structured RAG response tailored for executives.

    Fields:
      - answer: short, clear summary based strictly on retrieved context.
      - bullets: 3-6 key bullet points highlighting the most important aspects.
      - citations: free-form identifiers of the sources used (e.g., filenames).
    """
    answer: str = Field(
        ...,
        description="A short, clear answer based solely on context."
    )
    bullets: list[str] = Field(
        ...,
        description="List 3-6 key points of the summary."
    )
    citations: list[str] = Field(
        default_factory=list,
        description="Identifiers of sources used in the response (e.g. file name or title)."
    )


# -------------------------
# VectorStore & embedding
# -------------------------
# Configure the vector store backend (Chroma) and collection to use.
cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)

# High-level vector store manager wrapping the underlying implementation.
store = VectorstoreManager(config=cfg)

# Embedding manager shared across:
#   - document ingestion,
#   - query encoding,
#   - re-ranking logic.
embed_manager = EmbeddingManager(    
    provider="ollama",
    model_name=GLOBAL_SETTINGS.default_ollama_embed_model,
    assume_ollama_dim=1536,
)

# Re-ranker that refines initial similarity scores from the vector store.
# Score fusion combines base similarity with re-ranker scores for better ranking.
reranker = ReRanker(
    embedding_manager=embed_manager,
    config=ReRankerConfig(
        use_score_fusion=True,
        fusion_alpha=0.4,
        normalize="minmax",
        doc_batch_size=256,
    ),    
)

# Retriever responsible for:
#   - querying the vector store,
#   - returning an ordered list of relevant chunks.
retriever = RagRetriever(store, embed_manager)


# -------------------------
# Answerer config
# -------------------------
# Configuration for the RAG answerer:
#   - top_k: maximum number of chunks retrieved.
#   - min_score: minimum similarity threshold for keeping a chunk.
#   - re_rank_k: how many chunks pass through the re-ranker.
#   - max_context_chars: maximum concatenated context passed to the LLM.
cfg = AnswererConfig(
    top_k=10,
    min_score=0.15,
    re_rank_k=5,
    max_context_chars=12000,
)

# System-level instructions controlling STRICT RAG behavior.
cfg.system_instructions = default_rag_system_instruction()

# Template used to inject retrieved context into the system message for the LLM.
cfg.system_context_template = (
    "Use the following context to answer the user's question: {context}"
)


# -------------------------
# LLM: OLLAMA
# (Adapter should support generate_structured, e.g., via JSON mode + validation)
# -------------------------
# Create the LLM via the Intergrax adapter registry; here using Ollama backend.
llm = LLMAdapterRegistry.create(LLMProvider.OLLAMA)


# Conversational memory for multi-turn interactions and follow-up questions.
memory = ConversationalMemory()

# High-level RAG answerer:
#   - runs retrieval + re-ranking,
#   - builds prompts,
#   - optionally performs structured generation when output_model is provided.
answerer = RagAnswerer(
    retriever=retriever,
    llm=llm,
    reranker=reranker,
    config=cfg,
    memory=memory,    
)


# -------------------------
# QUESTION 1: structured response (output_model=ExecSummary)
# -------------------------
q1 = (
    "What unique advantages does Mooff have over its competitors? "
    "Describe in as much detail as possible. Use a minimum of 1,000 words. "
    "Use the passed output_model to generate the response."
)

print("QUESTION (structured):", q1)

# Run the RAG pipeline with a structured output model.
#   - summarize=False: we rely on ExecSummary instead of a generic summary.
#   - output_model=ExecSummary: forces the answerer to construct a typed object.
res1 = answerer.run(
    question=q1,
    stream=False,
    summarize=False,           # no generic summary; we have structured schema
    output_model=ExecSummary,  # returned under 'output_structure'
)

# res1["answer"]: free-form textual answer (if the pipeline still provides it).
print("ANSWER (text):", res1["answer"])

# res1["output_structure"]: instance of ExecSummary or None if parsing failed.
print("OUTPUT_STRUCTURE:", res1["output_structure"])

# Print compact list of sources in "source|page" or "source" form.
print(
    "SOURCES:",
    [
        f"{s.source}|{s.page}" if s.page else s.source
        for s in res1["sources"]
    ],
)
print()



QUESTION (structured): What unique advantages does Mooff have over its competitors? Describe in as much detail as possible. Use a minimum of 1,000 words. Use the passed output_model to generate the response.
ANSWER (text): No sufficiently relevant context fragments were found to answer the question.
OUTPUT_STRUCTURE: None
SOURCES: []

